In [57]:
%pip install nltk
%pip install tensorflow
%pip install yake
%pip install spacy
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [59]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

In [60]:
import spacy

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
import yake

In [63]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
spacy.cli.download('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [64]:
# Leitura e pré-processamento dos dados
def preprocess_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text = text.lower()  # Convertendo para minúsculas
    return text

In [65]:
# Definição dos hiperparâmetros
max_sequence_length = 1000
max_words = 10000
embedding_dim = 100

In [66]:
# Carregamento e pré-processamento dos dados
data = preprocess_data('texto.txt')

In [67]:
# Tokenização dos dados
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts([data])
word_index = tokenizer.word_index
total_words = len(word_index) + 1

In [68]:
# Criação das sequências de treinamento
input_sequences = []
for line in data.split('\n'):
    tokenized = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokenized)):
        n_gram_sequence = tokenized[:i+1]
        input_sequences.append(n_gram_sequence)

In [69]:
# Padding das sequências de entrada
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

In [70]:
# Separação dos dados de entrada e saída
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

In [71]:
# Criação dos vetores one-hot para as saídas
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [72]:
# Criação do modelo de rede neural
model = Sequential()
model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [73]:
# Treinamento do modelo
model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
15/15 [==============================] - 3s 80ms/step - loss: 5.4133
Epoch 2/100
15/15 [==============================] - 2s 137ms/step - loss: 5.0826
Epoch 3/100
15/15 [==============================] - 2s 156ms/step - loss: 4.9365
Epoch 4/100
15/15 [==============================] - 2s 161ms/step - loss: 4.8960
Epoch 5/100
15/15 [==============================] - 2s 152ms/step - loss: 4.8855
Epoch 6/100
15/15 [==============================] - 2s 149ms/step - loss: 4.8720
Epoch 7/100
15/15 [==============================] - 3s 190ms/step - loss: 4.8498
Epoch 8/100
15/15 [==============================] - 3s 175ms/step - loss: 4.8171
Epoch 9/100
15/15 [==============================] - 2s 166ms/step - loss: 4.7893
Epoch 10/100
15/15 [==============================] - 2s 161ms/step - loss: 4.7177
Epoch 11/100
15/15 [==============================] - 1s 83ms/step - loss: 4.6121
Epoch 12/100
15/15 [==============================] - 1s 82ms/step - loss: 4.4746
Epoch 13/100
15/

In [74]:
# Geração de texto a partir do modelo treinado
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word

    return seed_text

In [75]:
# Pré-processamento do texto a partir do texto gerado pelo modelo
def preprocess_text(text):
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]

    stop_words = set(stopwords.words('english'))
    words = [[word for word in sentence if word.lower() not in stop_words] for sentence in words]

    tagged_words = [pos_tag(sentence, lang='eng') for sentence in words]

    return tagged_words

In [76]:
# Reconhecimento de Entidades Nomeadas
def extract_named_entities(tagged_words):
    entities = []
    for sentence in tagged_words:
        text = ' '.join([word + '/' + tag for word, tag in sentence])
        doc = nlp(text)
        for entity in doc.ents:
            entities.append((entity.text, entity.label_))

    return entities

In [77]:
# Extração das palavras chave
def extract_keywords(text, num_keywords=20):
    # Tokenizar o texto em palavras
    tokens = nltk.word_tokenize(text)

    # Remover stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Criar uma matriz TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(tokens)])

    # Converter a matriz esparsa em uma matriz densa
    dense_matrix = tfidf_matrix.toarray()

    # Obter os índices das palavras com as maiores pontuações TF-IDF
    top_indices = dense_matrix.argsort()[0, ::-1][:num_keywords]

    # Obter as palavras-chave correspondentes aos índices
    feature_names = vectorizer.get_feature_names_out()
    keywords = [feature_names[idx] for idx in top_indices]

    return keywords

In [78]:
# Criação do resumo
def create_summary(text, num_sentences=3):
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalnum()]

    word_frequencies = FreqDist(words)

    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies:
                if len(sentence.split(' ')) < 30:
                    if sentence not in sentence_scores:
                        sentence_scores[sentence] = word_frequencies[word]
                    else:
                        sentence_scores[sentence] += word_frequencies[word]

    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    summary = ' '.join(summary_sentences)

    return summary

In [79]:
# Carrega recursos inglês do scypi
nlp = spacy.load("en_core_web_sm")

In [90]:
# Gerando o texto
# Parâmetros: seed para geração do texto, quantidade de palavras seguintes,
# modelo, máximo de sequência de palavras possíveis
generated_text = generate_text("In Brazil", 100, model, max_sequence_length)
display(generated_text)

"In Brazil in addition to political polarization and the elections 2018 was also marked by corruption scandals that continued to shake the country operation car wash initiated in 2014 revealed a vast scheme of embezzlement of public funds involving high ranking politicians and businessmen this operation had a profound impact on brazilian society generating outrage and a demand for changes in the political system system was considered one of party party party party party party party party party party party the workers' party party party party party party party party party party the workers' party party party party party party party party"

In [91]:
# Pré-processamento do texto gerado para extração de entidades nomeadas,
# palavras-chave e criação do resumo a partir do texto

tagged_words = preprocess_text(generated_text)
entities = extract_named_entities(tagged_words)
key_words = extract_keywords(generated_text)
summary = create_summary(generated_text)

In [92]:
# Imprime as entidades nomeadas
print("Entidades Nomeadas:")
for entity in entities:
    print(entity)

Entidades Nomeadas:
('Brazil', 'GPE')
('NNP', 'ORG')
('2018', 'DATE')
('VBN', 'ORG')
('NNS', 'ORG')
('2014', 'DATE')
('NNS', 'ORG')
('VBG', 'ORG')
('NNS', 'ORG')
('NNS', 'PERSON')
('VBG', 'ORG')
('NNS', 'ORG')
('NNS', 'ORG')


In [93]:
# Imprime as palavras-chave
print("\nPalavras-chave:")
for keyword in key_words:
    print(keyword)


Palavras-chave:
party
workers
system
operation
political
continued
funds
embezzlement
elections
demand
country
corruption
considered
high
changes
car
businessmen
brazilian
brazil
also


In [94]:
# Passo 12: Imprimir o resumo
print("\nResumo:")
display(summary)


Resumo:


''

In [96]:
# Pré-processamento do texto gerado para extração de entidades nomeadas,
# palavras-chave e criação do resumo a partir do documento de texto inicial

txt_tagged_words = preprocess_text(data)
txt_entities = extract_named_entities(txt_tagged_words)
txt_key_words = extract_keywords(data)
txt_summary = create_summary(data)

In [97]:
# Imprime as entidades nomeadas
print("Entidades Nomeadas:")
for entity in txt_entities:
    print(entity)

Entidades Nomeadas:
('JJ 2018', 'DATE')
('NNS', 'ORG')
('VBZ', 'ORG')
('NNS', 'ORG')
('NNS', 'ORG')
('society/NN', 'ORG')
('NNS', 'ORG')
('VBN', 'ORG')
('NNS', 'ORG')
('VBN', 'ORG')
('NNS', 'ORG')
('VBP', 'ORG')
('NNS', 'PERSON')
('NNS', 'ORG')
('VBN', 'ORG')
('NNS', 'ORG')
('VBN', 'ORG')
('NNS', 'ORG')
('NNS', 'ORG')
('VBG', 'ORG')
('NNS', 'ORG')
('NNS', 'ORG')
('VBP', 'ORG')
('NNS', 'ORG')
('VBN', 'ORG')
('NNS', 'PERSON')
('NNS', 'ORG')
('VBG', 'ORG')
('NNS', 'ORG')
('VBP', 'ORG')
('NNS', 'ORG')
('VBP', 'ORG')
('VBG', 'ORG')
('NNS', 'PERSON')
('NNS', 'ORG')
('VBP', 'ORG')
('NNS', 'ORG')
('2018', 'DATE')
('VBN', 'ORG')
('NNS', 'ORG')
('JJ 2014', 'DATE')
('NNS', 'ORG')
('VBG', 'ORG')
('NNS', 'ORG')
('VBG', 'ORG')
('brazil', 'GPE')
('VBP', 'ORG')
('NNS', 'ORG')
('VBP 2018', 'ORG')
('NNS', 'ORG')
('NNS', 'ORG')
('VBP', 'ORG')
('NNS', 'ORG')
('VBG', 'ORG')
('brazil', 'GPE')
('VBG', 'ORG')


In [98]:
# Imprime as palavras-chave
print("\nPalavras-chave:")
for keyword in txt_key_words:
    print(keyword)


Palavras-chave:
political
elections
corruption
country
impact
marked
debates
brazil
support
politics
future
social
candidates
one
bolsonaro
significant
polarization
events
jair
presidential


In [99]:
summary_doc = create_summary(data)
print("\nResumo do documento:")
display(summary_doc)


Resumo do documento:


'the results of these events and political actions had a lasting impact on brazil, shaping the political and social landscape in the subsequent years. the politics in brazil in 2018 were marked by a series of events and occurrences that had a significant impact on the country. it was a year of presidential elections and intense debates about the political, social, and economic future of the nation.'